<a href="https://stella-project.org/"><img align="right" width="100" src="doc/img/logo-st.JPG" /></a>
<br>
# STELLA Demo

This interactive demo guide walks you through the [STELLA](https://stella-project.org/) infrastructure using the STELLA [demo.yml](https://github.com/stella-project/stella-app/blob/master/demo.yml) docker-compose stack. It is designed to give a simple but comprehensive overview of the functionalities of the STELLA Living Lab infrastructure.

<br>

**Table of content:**
1. Setup
2. STELLA app
  - 2.1 Indexing documents
  - 2.2 Querying results
  - 2.3 Saving feedback
  - 2.4 End session:
3. STELLA server
  - 3.1 Server web interface:
  - 3.2 Authentication
  - 3.3 Saving feedback
  - 3.4 Retrieve feedback
  - 3.5 Retrieve information
  - 3.6 Export results

<br><br>

## 1. Setup
---
For an in depth setup guide please visit the [full documentation](https://github.com/stella-project/stella-documentation/wiki/Setup-guide-(local)). For this guide, the [demo.yml](https://github.com/stella-project/stella-app/blob/master/demo.yml) compose-stack is used. 

Before starting, please make sure:
1. `docker` and `docker-compose` are installed,
2. the repositorys [stella-app](https://github.com/stella-project/stella-app) and [stella-server](https://github.com/stella-project/stella-server) are cloned and
3. you have downloaded at least some of the provided [data](https://th-koeln.sciebo.de/s/OBm0NLEwz1RYl9N?path=%2F) to the `/data` directory.

If these requirements are satisfied, at most, minimal adjustments should be needed. While the infrastructure is intended as a distributed system, for this tutorial we will set up all components on one machine, using a single docker-compose stack. Any changes, if needed, take place in the [demo.yml](https://github.com/stella-project/stella-app/blob/master/demo.yml) file. If not freely available on your machine, make sure to change the ports `8000` and `8080`. 
Since indexing the data can be performance-intensive, depending on the amount of data, experimental systems and your system specifications, it can be necessary to disable the `BULK_INDEX` option to prevent auto indexing at system start.

Now you should be ready to start your system. This may take a while, so please be patient.

In [1]:
!docker-compose -f demo.yml up -d

Creating gesis_rec_precom ... 
Creating stella-app_db_1  ... 
Creating livivo_rank_precom ... 
Creating stella-app_db_server_1 ... 
Creating gesis_rec_pyserini     ... 
Creating livivo_rank_pyserini   ... 
Creating stella-app_web_1       ... mdone
Creating stella-app_app_1       ... mdone
ting stella-app_app_1       ... done

<br><br>

## 2. STELLA app:
---

The STELLA app connects the search interface, like [LIVIVO](https://www.livivo.de), [Gesis](https://www.gesis.org/en/home) or your own,  with the experimental ranking systems and is responsible for interleaving the result pages. For this demo we installed the ranking systems `livivo_rank_precom`, `livivo_rank_pyserini`, `gesis_rec_precom` and `gesis_rec_pyserini`, just to cover all use-cases. These four ranking systems, two baseline systems and two experimental systems are included in the [demo.yml](https://github.com/stella-project/stella-app/blob/master/demo.yml) file. <br>
To learn how to develop your own ranking system or to use a pre-computed run please visit the full [documentation](https://github.com/stella-project/stella-documentation/wiki).

For interaction and communication between systems different endpoints are available and described in the following:

In [1]:
STELLA_URL = 'http://0.0.0.0:8080'
STELLA_APP_API = 'http://0.0.0.0:8080/stella/api/v1/'

<br>

### 2.1 Indexing documents:
If `BULK_INDEX` is disabled while creating the docker-compose stack, systems can be indexed either through the [stella-app web ui](http://0.0.0.0:8080) or the stella-app endpoint.

**1. Web UI:**

You can reach the stella-app web UI via [http://0.0.0.0:8080/](http://0.0.0.0:8080/) and trigger indexing for specific systems.

**2. Endpoint:**
- `/index/<system_name>` let you index a specific system.

In [2]:
import requests

In [ ]:
SYSTEM_NAME = 'livivo_rank_pyserini'

r = requests.get(STELLA_URL + '/index/' + SYSTEM_NAME)
print(r.text)

- `/index/bulk` let you trigger the full bulk index.

In [ ]:
r = requests.get(STELLA_URL + '/index/bulk')
print(r.text)

Indexing may take a while. If it is done, you are ready for your first search.

<br>

### 2.2 Querying results:
For querying interleaved results you can use two endpoints, depending on the system type.

#### GET ranking
will return an interleaved ranking result, given a query string, from an experimental ranking system and the base system.

In [5]:
query = "influenza"

In [6]:
r = requests.get(STELLA_APP_API + "ranking?query=" + query)
ranking = r.json()

In [7]:
ranking

{'body': {'1': {'docid': 'AGRISUS201500178988', 'type': 'BASE'},
  '10': {'docid': 'NLM1303033R', 'type': 'EXP'},
  '11': {'docid': 'NLM101141746', 'type': 'EXP'},
  '12': {'docid': 'AGRISUS201500141906', 'type': 'BASE'},
  '13': {'docid': 'M10633731', 'type': 'EXP'},
  '14': {'docid': 'NLM101253340', 'type': 'BASE'},
  '15': {'docid': 'M8948994', 'type': 'EXP'},
  '16': {'docid': 'AGRISUS201500141840', 'type': 'BASE'},
  '17': {'docid': 'AGRISUS201600061219', 'type': 'BASE'},
  '18': {'docid': 'NLM35010190R', 'type': 'EXP'},
  '19': {'docid': 'AGRISUS201500142797', 'type': 'BASE'},
  '2': {'docid': 'NLM101123650', 'type': 'EXP'},
  '20': {'docid': 'NLM101253020', 'type': 'EXP'},
  '3': {'docid': 'NLM101253317', 'type': 'BASE'},
  '4': {'docid': 'NLM101508233', 'type': 'EXP'},
  '5': {'docid': 'NLM9104537', 'type': 'EXP'},
  '6': {'docid': 'AGRISUS201600058285', 'type': 'BASE'},
  '7': {'docid': 'M18118800', 'type': 'EXP'},
  '8': {'docid': 'AGRISUS201500178858', 'type': 'BASE'},
  '9'

#### GET recommendation
returns interleaved recommendations, given the id of a seed document, from an experimental system and the base system.

In [8]:
itemid = "gesis-ssoar-13114"

In [9]:
r = requests.get(STELLA_APP_API + "recommendation/datasets?itemid=" + itemid)
recommendation = r.json()

In [10]:
recommendation

{'body': {'1': {'docid': 'datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de701348',
   'type': 'BASE'},
  '2': {'docid': 'datasearch-httpservices-fsd-uta-fioai--oaifsd-uta-fiFSD2157e',
   'type': 'EXP'},
  '3': {'docid': 'datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de701349',
   'type': 'BASE'},
  '4': {'docid': 'datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de439108',
   'type': 'EXP'},
  '5': {'docid': 'datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de439107',
   'type': 'EXP'},
  '6': {'docid': 'datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de702532',
   'type': 'BASE'},
  '7': {'docid': 'datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de702533',
   'type': 'BASE'},
  '8': {'docid': 'datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de652472',
   'type': 'EXP'},
  '9': {'docid': 'ZA5368', 'type': 'BASE'}},
 'header': {'container': {'base': 'gesis_rec_pyserini',
   'exp': 'gesis_rec_precom'},
  'itemid': 'gesis-ssoar-13114',
  'page': 0,
  'rid': 12,
  'rpp': 10,
  'sid': '9f42ca44f4e643e5

<br>

### 2.3 Saving feedback:

These result lists will be clicked by the users and then returned to the app, including the feedback data, using the feedback endpoints. Multiple feedbacks can be combined into sessions.

For demo purposes, we use simulated ranking and recommendation click feedbacks.

In [11]:
import json

#### POST ranking feedback:

In [12]:
ranking_click_dict = {
     "1": {"docid": "AGRISUS201500178988", "type": "BASE", "clicked": True, "date": "2020-01-01 00:00:01"},
     "2": {"docid": "NLM101123650", "type": "EXP", "clicked": True, "date": "2020-01-01 00:00:02"},
     "3": {"docid": "NLM101508233", "type": "EXP", "clicked": False, "date": None},
     "4": {"docid": "NLM101253317", "type": "BASE", "clicked": False, "date": None},
     "5": {"docid": "NLM9104537", "type": "EXP", "clicked": False, "date": None},
     "6": {"docid": "AGRISUS201600058285", "type": "BASE", "clicked": False, "date": None},
     "7": {"docid": "AGRISUS201500178858", "type": "BASE", "clicked": False, "date": None},
     "8": {"docid": "M18118800", "type": "EXP", "clicked": False, "date": None},
     "9": {"docid": "NLM1303033R", "type": "EXP", "clicked": False, "date": None},
     "10": {"docid": "NLM101261447", "type": "BASE", "clicked": False, "date": None},
     "11": {"docid": "AGRISUS201500141906", "type": "BASE", "clicked": False, "date": None},
     "12": {"docid": "NLM101141746", "type": "EXP", "clicked": False, "date": None},
     "13": {"docid": "NLM101253340", "type": "BASE", "clicked": False, "date": None},
     "14": {"docid": "M10633731", "type": "EXP", "clicked": False, "date": None},
     "15": {"docid": "AGRISUS201500141840", "type": "BASE", "clicked": False, "date": None},
     "16": {"docid": "M8948994", "type": "EXP", "clicked": False, "date": None},
     "17": {"docid": "AGRISUS201600061219", "type": "BASE", "clicked": False, "date": None},
     "18": {"docid": "NLM35010190R", "type": "EXP", "clicked": False, "date": None},
     "19": {"docid": "AGRISUS201500142797", "type": "BASE", "clicked": False, "date": None},
     "20": {"docid": "NLM101253020", "type": "EXP", "clicked": False, "date": None},
     }

In [13]:
ranking_payload = {
    'start': "2020-01-01 00:00:00",
    'end': "2020-01-01 00:01:00",
    'interleave': True,
    'clicks': json.dumps(ranking_click_dict)
}

In [14]:
ranking_id = ranking.get('header').get('rid')
r_post = requests.post(STELLA_APP_API + "ranking/" + str(ranking_id) + "/feedback", data=ranking_payload)
print(r_post.status_code)

201


#### POST recommendation feedback:

In [15]:
recommendation_click_dict = {
     "1": {"docid": "datasearch-httpservices-fsd-uta-fioai--oaifsd-uta-fiFSD2157e", "type": "EXP", "clicked": True, "date": "2020-01-01 00:00:01"},
     "2": {"docid": "datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de701348", "type": "BASE", "clicked": False, "date": None},
     "3": {"docid": "datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de439108", "type": "EXP", "clicked": False, "date": None},
     "4": {"docid": "datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de701349", "type": "BASE", "clicked": False, "date": None},
     "5": {"docid": "datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de439107", "type": "EXP", "clicked": False, "date": None},
     "6": {"docid": "datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de702532", "type": "BASE", "clicked": True, "date": "2020-01-01 00:00:06"},
     "7": {"docid": "datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de702533", "type": "BASE", "clicked": False, "date": None},
     "8": {"docid": "datasearch-httpwww-da-ra-deoaip--oaioai-da-ra-de652472", "type": "EXP", "clicked": False, "date": None},
     "9": {"docid": "ZA5368", "type": "BASE", "clicked": False, "date": "2020-01-01 00:00:09"}
    }

In [16]:
recommendation_payload = {
    'start': "2020-01-01 00:00:00",
    'end': "2020-01-01 00:01:00",
 'interleave': True,
 'clicks': json.dumps(recommendation_click_dict)
}

In [17]:
recommendation_id = recommendation.get('header').get('rid')
r_post = requests.post(STELLA_APP_API + "recommendation/" + str(recommendation_id) + "/feedback", data=recommendation_payload)
print(r_post.status_code)

201


All feedbacks will be regularly transfered to the STELLA server using the STELLA server api described in the next part.

<br>

### 2.4 End session:

If you need to end a session manually you can call a dedicated endpoint for that.

In [18]:
r = requests.put(STELLA_APP_API + '/sessions/' + str(1) + '/exit')

<br><br>

## 3. STELLA server:
---
The STELLA server gathers all user feedback and lets you administrate all systems. Furthermore, it is the main interaction point for participants, submitting and monitoring their systems. To interact with the STELLA server authentication is needed. Therefore we have to generate an authentication token first. Next, we can send and retrieve feedbacks. 
Besides that, the STELLA server provides endpoints to get information about sites, participants and systems. Finally, the `export` endpoint can be used to retrieve full feedback data on a system.

In [19]:
STELLA_SERVER_API = 'http://0.0.0.0:8000/stella/api/v1'
site = "LIVIVO"
system = "livivo_rank_pyserini"

<br>

### 3.1 Server web interface:

You can reach the stella-server web UI via [http://0.0.0.0:8000/](http://0.0.0.0:8000/) and log in with the username and password specified in the [demo.yml](https://github.com/stella-project/stella-app/blob/master/demo.yml) docker-compose file. By default, `admin@stella-project.org` is specified as admin username and `pass` as password. Depending on the type of user (admin, site, or participant) different functionalities are available:
- The [Dashboard](http://0.0.0.0:8000/dashboard) tab provides statistics about the performance of individual systems.
- The [Systems](http://0.0.0.0:8000/systems) tab lists all registered systems and additional information. New systems can be registered here and feedback data of existing as well as all systems can be downloaded here. 
- The [Administration](http://0.0.0.0:8000/administration) tab, available to all administrators helps to update the STELLA app.

<br>

### 3.2 Authentication:

The STELLA server is secured by a username and a password, specified in the [demo.yml](https://github.com/stella-project/stella-app/blob/master/demo.yml) docker-compose file. To securely communicate with the server's API we need to create an authentication token.

In [20]:
r = requests.post(STELLA_SERVER_API + '/tokens', auth=('livivo@stella-project.org', 'pass'))
r_json = json.loads(r.text)
token = r_json.get('token')

<br>

### 3.3 Saving feedback:

Collected feedback from the STELLA App is sent to the STELLA Server. Therefore dedicated endpoints are provided. First, the site identifier of the search website is needed.

In [21]:
r = requests.get(STELLA_SERVER_API + '/sites/' + site, auth=(token, ''))
r_json = r.json()
site_id = r_json.get('id')
print("Site ID:", str(site_id))

Site ID: 5


#### POST ranking session:
Using the site identifier we can create a new session providing a start and end date as well as the site user.

In [22]:
payload = {
    'site_user': '123.123.123.123',
    'start': "2020-01-01 00:00:00",
    'end': "2020-01-01 00:01:00",
    'system_ranking': system
}

In [23]:
r = requests.post(STELLA_SERVER_API + '/sites/' + str(site_id) + '/sessions', data=payload, auth=(token, ''))
session_id = r.json()["session_id"]
print("Session ID:", str(session_id))

Session ID: 3


#### POST ranking feedback:
To send a full feedback to the STELLA Server we use this session identivier and the feedback data specifies in section 2.3 as `ranking_payload`.

In [24]:
r = requests.post(STELLA_SERVER_API + '/sessions/' + str(session_id) + '/feedbacks', data=ranking_payload, auth=(token, ''))
r_json = r.json()
feedback_id = r_json['feedback_id']
print("Feedback ID:", str(feedback_id))

Feedback ID: 3


#### POST ranking:
Given the feedback identifier, we can add a ranking to the feedback.

In [25]:
items = {
    "1": "doc1",
    "2": "doc2",
    "3": "doc3",
    "4": "doc4",
    "5": "doc5",
    "6": "doc6",
    "7": "doc7",
    "8": "doc8",
    "9": "doc9",
    "10": "doc10"
}

In [26]:
payload = {
    'q': 'query goes here!',
    'q_date': "2020-01-01 00:00:00",
    'q_time': 300,
    'num_found': 10,
    'page': 1,
    'rpp': 10,
    'items': json.dumps(items)
}

In [27]:
r = requests.post(STELLA_SERVER_API + '/feedbacks/' + str(feedback_id) + '/rankings', data=payload, auth=(token, ''))
r_json = r.json()
ranking_id = r_json['ranking_id']
print("Ranking ID:", str(ranking_id))

Ranking ID: 7


<br>

### 3.4 Retrieve feedback:
The endpoints `sessions`, `rankings`, and `feedbacks` can be used to retrieve the data using `get`. 

#### GET session:

In [28]:
r = requests.get(STELLA_SERVER_API + '/sessions/' + str(session_id), auth=(token, ''))
print(r.text)

{
  "end": "2020-01-01 00:01:00", 
  "id": 3, 
  "site_id": 5, 
  "site_user": "123.123.123.123", 
  "start": "2020-01-01 00:00:00", 
  "system_ranking": 3, 
  "system_recommendation": null
}



#### GET ranking:

In [29]:
r = requests.get(STELLA_SERVER_API + '/rankings/' + str(ranking_id), auth=(token, ''))
print(r.text)

{
  "feedback_id": 3, 
  "items": {
    "1": "doc1", 
    "10": "doc10", 
    "2": "doc2", 
    "3": "doc3", 
    "4": "doc4", 
    "5": "doc5", 
    "6": "doc6", 
    "7": "doc7", 
    "8": "doc8", 
    "9": "doc9"
  }, 
  "num_found": 10, 
  "page": 1, 
  "participant_id": 3, 
  "q": "query goes here!", 
  "q_date": "2020-01-01 00:00:00", 
  "q_time": 300, 
  "rpp": 10, 
  "session_id": 3, 
  "site_id": 5, 
  "system_id": 3, 
  "type": "RANK"
}



#### GET feedback:

In [30]:
r = requests.get(STELLA_SERVER_API + '/feedbacks/' + str(feedback_id), auth=(token, ''))
print(r.text)

{
  "clicks": {
    "1": {
      "clicked": true, 
      "date": "2020-01-01 00:00:01", 
      "docid": "AGRISUS201500178988", 
      "type": "BASE"
    }, 
    "10": {
      "clicked": false, 
      "date": null, 
      "docid": "NLM101261447", 
      "type": "BASE"
    }, 
    "11": {
      "clicked": false, 
      "date": null, 
      "docid": "AGRISUS201500141906", 
      "type": "BASE"
    }, 
    "12": {
      "clicked": false, 
      "date": null, 
      "docid": "NLM101141746", 
      "type": "EXP"
    }, 
    "13": {
      "clicked": false, 
      "date": null, 
      "docid": "NLM101253340", 
      "type": "BASE"
    }, 
    "14": {
      "clicked": false, 
      "date": null, 
      "docid": "M10633731", 
      "type": "EXP"
    }, 
    "15": {
      "clicked": false, 
      "date": null, 
      "docid": "AGRISUS201500141840", 
      "type": "BASE"
    }, 
    "16": {
      "clicked": false, 
      "date": null, 
      "docid": "M8948994", 
      "type": "EXP"
    }, 
    "1

<br>

### 3.5 Retrieve information:
In addition, user-centered endpoints lite `sites` and `participants` provide information like the number of sessions at a site or the systems provided by a user.

#### GET sessions of site:

In [31]:
r = requests.get(STELLA_SERVER_API + '/sites/' + str(site_id) + '/sessions', auth=(token, ''))
r.json()[0]

{'end': '2021-05-31 16:40:53',
 'id': 1,
 'site_id': 5,
 'site_user': 'unknown',
 'start': '2021-05-31 16:40:47',
 'system_ranking': 2,
 'system_recommendation': None}

#### GET systems of site:

In [ ]:
# ERROR
r = requests.get(STELLA_SERVER_API + '/sites/' + str(site_id) + '/systems', auth=(token, ''))
print(r.text)

#### GET Participant information:

In [33]:
r = requests.get(STELLA_SERVER_API + '/participants/' + str(3) + '/systems', auth=(token, ''))
print(r.text)

[
  {
    "id": 2, 
    "name": "livivo_rank_precom", 
    "participant_id": 3, 
    "type": "RANK", 
    "url": "https://github.com/stella-project/livivo_rank_precom"
  }, 
  {
    "id": 3, 
    "name": "livivo_rank_pyserini", 
    "participant_id": 3, 
    "type": "RANK", 
    "url": "https://github.com/stella-project/livivo_rank_pyserini"
  }, 
  {
    "id": 4, 
    "name": "livivo_rank_pyterrier", 
    "participant_id": 3, 
    "type": "RANK", 
    "url": "https://github.com/stella-project/livivo_rank_pyterrier"
  }
]



#### GET system ID:

In [34]:
r = requests.get(STELLA_SERVER_API + '/system/id/' + str(system), auth=(token, ''))
system_id = r.json()["system_id"]
print("System ID:", str(system_id))

System ID: 3


<br>

### 3.6 Export results:
By calling the `export` endpoint on a system identifier, all results for a system are combined.

#### GET full results for system:

In [35]:
r = requests.get(STELLA_SERVER_API + '/system/' + str(system_id) + '/export', auth=(token, ''))
print(r.text)

{
  "Results": [
    {
      "clicks": {
        "1": {
          "clicked": true, 
          "date": "2020-01-01 00:00:01", 
          "docid": "AGRISUS201500178988", 
          "type": "BASE"
        }, 
        "10": {
          "clicked": false, 
          "date": null, 
          "docid": "NLM101261447", 
          "type": "BASE"
        }, 
        "11": {
          "clicked": false, 
          "date": null, 
          "docid": "AGRISUS201500141906", 
          "type": "BASE"
        }, 
        "12": {
          "clicked": false, 
          "date": null, 
          "docid": "NLM101141746", 
          "type": "EXP"
        }, 
        "13": {
          "clicked": false, 
          "date": null, 
          "docid": "NLM101253340", 
          "type": "BASE"
        }, 
        "14": {
          "clicked": false, 
          "date": null, 
          "docid": "M10633731", 
          "type": "EXP"
        }, 
        "15": {
          "clicked": false, 
          "date": null, 
       